In [8]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [12]:
stops = gpd.read_file('zip://../../rawdata/bus/bus_stops_nyc_nov2018.zip')
routes = gpd.read_file('zip://../../rawdata/bus/bus_routes_nyc_nov2018.zip')

In [13]:
routes

,route_id,dir_id,route_dir,route_shor,route_long,color,geometry
0,B100,0,B100_0,B100,Mill Basin - Midwood,B933AD,(LINESTRING (1000986.095828866 160488.73216041...
1,B100,1,B100_1,B100,Mill Basin - Midwood,B933AD,(LINESTRING (1007559.40884605 160383.752572090...
2,B103,0,B103_0,B103,Canarsie - Downtown Brooklyn Ltd,FAA61A,(LINESTRING (998864.4328709817 169743.44178691...
3,B103,1,B103_1,B103,Canarsie - Downtown Brooklyn Ltd,FAA61A,(LINESTRING (1014370.851199239 176102.46997003...
4,BX23,0,BX23_0,Bx23,Pelham Bay - Co-Op City,B933AD,(LINESTRING (1032121.55002699 249858.501465256...
5,BX23,1,BX23_1,Bx23,Pelham Bay - Co-Op City,B933AD,(LINESTRING (1030184.292760497 258556.27393541...
6,Q06,0,Q06_0,Q6,Sutphin Boulevard,B933AD,(LINESTRING (1049341.268338914 177120.03690117...
7,Q06,1,Q06_1,Q6,Sutphin Boulevard,B933AD,(LINESTRING (1046902.851992052 179892.00229999...
8,Q07,0,Q07_0,Q7,Rockaway Boulevard,6CBE45,(LINESTRING (1043033.159817661 184536.27709996...
9,Q07,1,Q07_1,Q7,Rockaway Boulevard,6CBE45,(LINESTRING (1041172.672955742 177194.98839209...
